In [1]:
import os
from data_gatherer.data_gatherer import DataGatherer
import pandas as pd

In [2]:
input_file = "scripts/exp_input/REV.txt"

In [3]:
model_name = "gemini-2.0-flash"  # "gemini-2.0-flash" or "gpt-4o-mini"
prompt = "GPT_FewShot"  # "GPT_from_full_input_Examples" or "GPT_FewShot"
FDR = False
semantic_retrieval = True
section_filter= None

In [4]:
# write list to a text file
with open(input_file, 'r') as f:
    pmcids = f.read().splitlines()[:20]

print("Number of PMCIDs:", len(pmcids))

Number of PMCIDs: 20


In [5]:
dg = DataGatherer(
    llm_name=model_name, 
    log_level='INFO', 
    process_entire_document=FDR, 
    driver_path='../Firefox/geckodriver', 
    save_to_cache=False, 
    load_from_cache=False,
    full_output_file="scripts/output/result.csv"
) #, save_dynamic_prompts=True

data_gatherer.py - line 329 - INFO - Setting up data fetcher...
data_fetcher.py - line 102 - INFO - Backup data store initialized: 1313 publications, valid: True
data_gatherer.py - line 355 - INFO - Data fetcher setup completed.
data_gatherer.py - line 108 - INFO - DataGatherer orchestrator initialized. Extraction Model: gemini-2.0-flash


In [6]:
# These are the problematic urls for information extraction for GEMINI
#urls = ["https://pmc.ncbi.nlm.nih.gov/articles/PMC9314356", "https://pmc.ncbi.nlm.nih.gov/articles/PMC4318527/", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11929800/", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11032436/", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10802452/", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8565335/","https://pmc.ncbi.nlm.nih.gov/articles/PMC7778917/"]
# v1 = "https://pmc.ncbi.nlm.nih.gov/articles/PMC7029360/"
# v2 = "https://pmc.ncbi.nlm.nih.gov/articles/PMC8006362"
#urls = ["https://pmc.ncbi.nlm.nih.gov/articles/PMC9710693/"]

# These are the problematic urls for information extraction for GPT - (2025 Aug 29) GPT gets stuck for some reason after 50 calls or so...
#urls = ["https://pmc.ncbi.nlm.nih.gov/articles/PMC11240079", "https://pmc.ncbi.nlm.nih.gov/articles/PMC5752539","https://pmc.ncbi.nlm.nih.gov/articles/PMC7032692", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10547713", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8102856", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8055881", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8226229", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10769298", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11659981", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8082263", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8565335", "https://pmc.ncbi.nlm.nih.gov/articles/PMC9094742", "https://pmc.ncbi.nlm.nih.gov/articles/PMC7029360", "https://pmc.ncbi.nlm.nih.gov/articles/PMC6289083", "https://pmc.ncbi.nlm.nih.gov/articles/PMC7658217", "https://pmc.ncbi.nlm.nih.gov/articles/PMC9780309","https://pmc.ncbi.nlm.nih.gov/articles/PMC3788619", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8859891", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10680627", "https://pmc.ncbi.nlm.nih.gov/articles/PMC6323985", "https://pmc.ncbi.nlm.nih.gov/articles/PMC8131595", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11420198", "https://pmc.ncbi.nlm.nih.gov/articles/PMC9915613", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10238095", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10802452" , "https://pmc.ncbi.nlm.nih.gov/articles/PMC11032436", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10329279", "https://pmc.ncbi.nlm.nih.gov/articles/PMC10836119",  "https://pmc.ncbi.nlm.nih.gov/articles/PMC9280291", "https://pmc.ncbi.nlm.nih.gov/articles/PMC11661334", "https://pmc.ncbi.nlm.nih.gov/articles/PMC5161470", "https://pmc.ncbi.nlm.nih.gov/articles/PMC4339277"]

#out = dg.process_articles(urls, semantic_retrieval=semantic_retrieval, section_filter=section_filter, prompt_name=prompt)

#out
#out['https://pmc.ncbi.nlm.nih.gov/articles/PMC9710693']

In [7]:
combined_df = dg.run(input_file=pmcids, semantic_retrieval=semantic_retrieval, section_filter=section_filter, prompt_name=prompt)

data_gatherer.py - line 696 - INFO - #0 function call: self.process_url(https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10418516)
data_gatherer.py - line 421 - INFO - Processing URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10418516
data_fetcher.py - line 288 - INFO - API detected: PMC
data_fetcher.py - line 296 - INFO - Creating EntrezFetcher with backup support
data_fetcher.py - line 102 - INFO - Backup data store initialized: 1313 publications, valid: True
data_fetcher.py - line 834 - INFO - Raw_data_format: XML
data_gatherer.py - line 429 - INFO - Type of data_fetcher EntrezFetcher
data_fetcher.py - line 118 - INFO - Found PMC10418516 in backup data store (format: HTML)
data_fetcher.py - line 860 - INFO - Found PMC10418516 in local backup data (fast path, format: HTML)
html_retriever.py - line 254 - INFO - Checking if HTML data is complete
html_retriever.py - line 213 - INFO - Checking for data_availability_sections section in raw HTML data.
html_retriever.py - line 190 - INFO - L

Embedding corpus of 226 documents using sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/8 [00:00<?, ?it/s]

embeddings_retriever.py - line 169 - INFO - Searching for top-2 passages similar to the query by embeddings.
embeddings_retriever.py - line 151 - INFO - Computing L2 distances using numpy.
html_parser.py - line 935 - INFO - HTML semantic retrieval completed: found 2 relevant sections
base_parser.py - line 258 - INFO - Function_call: extract_datasets_info_from_content(...)
prompt_manager.py - line 33 - INFO - Loading prompt: GPT_FewShot from user_prompt_dir: None, subdir: 
base_parser.py - line 1205 - INFO - Expected string but got list. Converting list to string.
base_parser.py - line 266 - INFO - Content length: 6631
base_parser.py - line 1205 - INFO - Expected string but got list. Converting list to string.
base_parser.py - line 283 - INFO - Prompt messages total length: 2007 tokens
base_parser.py - line 289 - INFO - Prompt ID: gemini-2.0-flash-0-b132cda2e05c2af6bb2e476468d9126b189a2025201870cc4360671186611012
base_parser.py - line 1205 - INFO - Expected string but got list. Converti

Embedding time: 1.67s (0.007s per chunk)
Corpus embedding completed. Shape: (226, 384)
Chunking results: 226 original documents → 226 embedded chunks


...

Embedding corpus of 220 documents using sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

embeddings_retriever.py - line 169 - INFO - Searching for top-2 passages similar to the query by embeddings.
embeddings_retriever.py - line 151 - INFO - Computing L2 distances using numpy.
html_parser.py - line 935 - INFO - HTML semantic retrieval completed: found 2 relevant sections
base_parser.py - line 258 - INFO - Function_call: extract_datasets_info_from_content(...)
prompt_manager.py - line 33 - INFO - Loading prompt: GPT_FewShot from user_prompt_dir: None, subdir: 
base_parser.py - line 1205 - INFO - Expected string but got list. Converting list to string.
base_parser.py - line 266 - INFO - Content length: 7733
base_parser.py - line 1205 - INFO - Expected string but got list. Converting list to string.
base_parser.py - line 283 - INFO - Prompt messages total length: 2283 tokens
base_parser.py - line 289 - INFO - Prompt ID: gemini-2.0-flash-0-5f106719485ef2951540c7150644dc9236f7b4b8033dfaf918a163f772d23154
base_parser.py - line 1205 - INFO - Expected string but got list. Converti

Embedding time: 1.56s (0.007s per chunk)
Corpus embedding completed. Shape: (220, 384)
Chunking results: 220 original documents → 220 embedded chunks


base_parser.py - line 931 - INFO - Resolved URL: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE125101
base_parser.py - line 885 - INFO - Pattern: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=, resolved_dataset_page: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE125101
base_parser.py - line 887 - INFO - Link matches the pattern https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc= of resolved_dataset_page.
base_parser.py - line 518 - INFO - Final schema validation vals: GSE125101, geo, https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE125101
base_parser.py - line 431 - INFO - Extracted dataset: {'dataset_identifier': 'GSE125101', 'data_repository': 'geo', 'dataset_webpage': 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE125101', 'citation_type': 'n/a'}
base_parser.py - line 347 - INFO - Processing dataset: {'dataset_identifier': 'PXD012405', 'data_repository': 'PRIDE', 'dataset_webpage': 'https://www.ebi.ac.uk/pride/archive/projects/PXD012405', 'dataset_i

In [8]:
#combined_df = pd.read_csv("scripts/output/result.csv")
combined_df.head()

,dataset_identifier,data_repository,dataset_webpage,citation_type,access_mode,link,title,file_info,description,source_section,...,file_extension,a_attr_href,a_attr_class,a_attr_data-ga-action,a_attr_target,a_attr_rel,context_description,source_url,pub_title,raw_data_format
0,GSE207091,geo,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,n/a,Complex download,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,Regulatory Effects of ABA and GA on the Expres...,HTML
1,PXD044299,www.ebi.ac.uk,https://www.ebi.ac.uk/pride/archive/projects/P...,n/a,Complex download,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,Regulatory Effects of ABA and GA on the Expres...,HTML
2,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.3390/ijms241512380,10.3390/ijms241512380,n/a,n/a,n/a,...,None,https://doi.org/10.3390/ijms241512380,usa-link usa-link--external,click_feat_suppl,_blank,noopener noreferrer,,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,Regulatory Effects of ABA and GA on the Expres...,HTML
3,NaN,NaN,NaN,NaN,NaN,/articles/instance/10418516/bin/ijms-24-12380-...,Click here for additional data file.,"(420.9KB, zip)",n/a,ijms-24-12380-s001,...,zip,/articles/instance/10418516/bin/ijms-24-12380-...,usa-link,click_feat_suppl,NaN,NaN,,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,Regulatory Effects of ABA and GA on the Expres...,HTML
4,GSE116943,geo,https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,n/a,Complex download,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,BRCA2 deficiency instigates cGAS-mediated infl...,HTML


In [11]:
#ground_truth = pd.read_parquet("scripts/exp_input/dataset_citation_records_Table.parquet")
ground_truth = pd.read_parquet("scripts/output/gold/dataset_citation_records_Table.parquet")
#ground_truth = df

In [12]:
ground_truth.head()
# rename colum synid to 'identifier'
#ground_truth = ground_truth.rename(columns={'synid': 'identifier'})

,identifier,repository,citing_publication_link,citation_record_source,citation_record_from_doi,doi,pmcid
0,PXD059466,PRIDE,https://dx.doi.org/10.1038/S41467-025-56720-1,proteomexchange_search.tsv,1,10.1038/S41467-025-56720-1,
1,PXD051312,PRIDE,https://dx.doi.org/10.6019/PXD051312,proteomexchange_search.tsv,1,10.6019/PXD051312,
2,PXD051312,PRIDE,https://dx.doi.org/10.1002/prca.202400095,proteomexchange_search.tsv,1,10.1002/prca.202400095,
3,PXD051312,PRIDE,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,proteomexchange_search.tsv,0,,PMC11895760
4,PXD054431,PRIDE,https://dx.doi.org/10.17159/SAJS.2025/18571,proteomexchange_search.tsv,1,10.17159/SAJS.2025/18571,


In [13]:
def evaluate_performance(predict_df, ground_truth, orchestrator, false_positives_file, false_negatives_file=None):
    """ Evaluates dataset extraction performance using precision, recall, and F1-score. """

    recall_list, false_positives_output, false_negatives_output = [], [], []
    total_precision, total_recall, num_sources = 0, 0, 0

    for source_page in predict_df['source_url'].unique():
        #pub_id = source_page.split('/')[-1].lower()
        pub_id = source_page
        
        orchestrator.logger.info(f"Evaluating pub_id: {pub_id}")
        #gt_data = ground_truth[ground_truth['pmcid'].str.lower() == pub_id.lower()]  # extract ground truth
        gt_data = ground_truth[ground_truth['citing_publication_link'].str.lower() == pub_id.lower()]  # extract ground truth

        gt_datasets = set()
        for dataset_string in gt_data['identifier'].dropna().str.lower():
            gt_datasets.update(dataset_string.split(','))  # Convert CSV string into set of IDs

        orchestrator.logger.info(f"# of elements in gt_data: {len(gt_data)}. Element IDs: {gt_datasets}")

        num_sources += 1

        # Extract evaluation datasets for this source page
        eval_data = predict_df[predict_df['source_url'] == source_page]
        eval_datasets = set(eval_data['dataset_identifier'].dropna().str.lower())
        # Remove invalid entries
        eval_datasets.discard('n/a')
        eval_datasets.discard('')

        orchestrator.logger.info(f"Evaluation datasets: {eval_datasets}")

        # Handle cases where both ground truth and evaluation are empty
        if not gt_datasets and not eval_datasets:
            orchestrator.logger.info("No datasets in both ground truth and evaluation. Perfect precision and recall.")
            total_precision += 1
            total_recall += 1
            continue

        # Match Extraction Logic
        matched_gt, matched_eval = set(), set()

        # Exact Matches
        exact_matches = gt_datasets & eval_datasets  # Intersection of ground truth and extracted datasets
        matched_gt.update(exact_matches)
        matched_eval.update(exact_matches)

        # Partial Matches (Aliased Identifiers)
        for eval_id in eval_datasets - matched_eval:
            for gt_id in gt_datasets - matched_gt:
                if eval_id in gt_id or gt_id in eval_id:  # Partial match or alias
                    orchestrator.logger.info(f"Partial or alias match found: eval_id={eval_id}, gt_id={gt_id}")
                    matched_gt.add(gt_id)
                    matched_eval.add(eval_id)
                    break  # Stop once matched

        # **False Positives (Unmatched extracted datasets)**
        FP = eval_datasets - matched_eval
        false_positives_output.extend([false_p, pub_id] for false_p in FP)

        # **False Negatives (Unmatched ground truth datasets)**
        FN = gt_datasets - matched_gt
        false_negatives_output.extend((FN, pub_id)) if len(FN) > 0 else None

        # **Precision and Recall Calculation**
        true_positives = len(matched_gt)
        false_positives = len(FP)
        false_negatives = len(FN)

        precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
        recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

        orchestrator.logger.info(f"Precision for {source_page}: {precision}")
        orchestrator.logger.info(f"Recall for {source_page}: {recall}")

        if recall == 0:
            recall_list.append(source_page)

        # Accumulate totals
        total_precision += precision
        total_recall += recall

    # **Compute Overall Metrics**
    average_precision = total_precision / num_sources if num_sources > 0 else 0
    average_recall = total_recall / num_sources if num_sources > 0 else 0
    f1_score = (
        2 * (average_precision * average_recall) / (average_precision + average_recall)
        if (average_precision + average_recall) > 0
        else 0
    )

    orchestrator.logger.info(f"\nPerformance evaluation completed for {num_sources} source pages.")

    # **Save false positives**
    with open(false_positives_file, 'w') as f:
        for item in false_positives_output:
            f.write("%s\n" % item)

    if false_negatives_file:
        with open(false_negatives_file, 'w') as f:
            for item in false_negatives_output:
                f.write("%s\n" % item)

    return {
        "average_precision": average_precision,
        "average_recall": average_recall,
        "f1_score": f1_score
    }

In [14]:
#results = evaluate_performance(out, ground_truth, dg, "output/false_positives.txt")
# PMC4318527, 

In [15]:
results = evaluate_performance(combined_df, ground_truth, dg, "scripts/output/false_positives.txt", "scripts/output/false_negatives.txt")

3090732644.py - line 11 - INFO - Evaluating pub_id: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10418516
3090732644.py - line 19 - INFO - # of elements in gt_data: 2. Element IDs: {'pxd044299', 'gse207091'}
3090732644.py - line 30 - INFO - Evaluation datasets: {'gse207091', 'pxd044299'}
3090732644.py - line 72 - INFO - Precision for https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10418516: 1.0
3090732644.py - line 73 - INFO - Recall for https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10418516: 1.0
3090732644.py - line 11 - INFO - Evaluating pub_id: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6327059
3090732644.py - line 19 - INFO - # of elements in gt_data: 2. Element IDs: {'pxd007253', 'gse116943'}
3090732644.py - line 30 - INFO - Evaluation datasets: {'prjna299537', 'pxd007253', 'gse116943'}
3090732644.py - line 72 - INFO - Precision for https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6327059: 0.6666666666666666
3090732644.py - line 73 - INFO - Recall for https://www.ncbi.nlm.nih.gov/pmc/ar

In [16]:
results

{'average_precision': 0.8733333333333333,
 'average_recall': 0.925,
 'f1_score': 0.8984244670991658}